# Scikit-learn Regression - Cross Validation
작성자: 고정훈

## Algorithm and Hyper-parameter Selection

앞서 scikit-learn을 이용한 regression의 기본적인 흐름을 설명드렸습니다. Random Forest를 모델로 사용했는데, 그 이유는 Hyper-parameter를 많이 Tuning하지 않아도 언제나 준수한 결과를 주기 때문입니다. Random Forest에서 영 성적이 좋지 않은 데이터는 다른 알고리즘을 사용하더라도 만족할 만한 성적이 나오지 않을 가능성이 매우 높습니다. 그래서 처음 시도해보는 Benchmark로 Random Forest를 많이 사용합니다.

그렇지만 Random Forest가 최선의 알고리즘이라는 것은 아닙니다. 데이터에 따라 Neural Network이 좋을 수도 Support Vector Machine이 좋을 수도, Lasso가 좋을 수도 있습니다. scikit-learn에서는 상당히 많은 Regression 알고리즘을 제공합니다. 이중에 무엇이 데이터와 궁합이 잘 맞을지는 해보지 않고서는 알 수 없습니다. 그리고 알고리즘을 선택할 때 정합성 뿐만 아니라 계산 시간까지 고려해야 한다면 문제는 더 복잡합니다. 

알고리즘만 고른다고 끝은 아닙니다. 어떤 알고리즘은 모델의 세부 사항을 결정하는 Hyper-parameter를 제대로 tuning 하지 않으면 실망스러운 성적이 나오기 때문입니다. 

1. Algorhtm 선택 
2. Hyper-parameter 조정

Algorithm과 Hyper-parameter를 동시에 최적화 하는 문제를 CASH(Combined Algorithm Selection and Hyper-parameter Optimization) Problem이라고 부르기도 합니다. 

## Cross Validation

이전 글에서 Training Set과 Test Set을 쪼개서 Training Set으로 학습을 하고 Test Set으로 Test Score를 계산해봤습니다. 
Algorithm과 Hyper-parameter를 고를 때는 학습한 데이터가 아니라 학습하지 않은 데이터에 대한 성적으로 평가해야 합니다.

이때 학습에 사용하지 않은 데이터의 역할은 아래 두 가지 입니다. 

1. 어떤 알고리즘 혹은 Hyper-parameter가 최선인지 선택하는 기준.
2. 실제 상황에 적용했을 때의 예상 성능. 

실제 상황에 적용했을 때의 예상 성능이 가장 높을 것 같은 알고리즘과 Hyper-parameter를 선택해야 하기 때문에 사실 1번과 2번은 같은 얘기입니다. 다만 경우에 따라서 1번 용으로 사용하는 set을 validation set, 2번 용으로 사용하는 set을 test set으로 나눠서 취급하기도 합니다. 

여기서는 심플하게 설명드리기 위해 둘을 하나로 취급하고 validation set이라고 부르겠습니다. 

### K-Fold Cross Validataion

Trainin/Test 혹은 Trainng/Validation set을 한 번만 나누어서는 정확한 generalization 성능을 측정할 수 없습니다. 여기서 generalization 성능이란 학습에 사용하지 않은 데이터에 대한 성능을 말합니다. 

Training/Validation set이 어떻게 나눠지느냐에 따라 $R^2$ 혹은 MSE 등의 편차가 크게 나타날 수 있습니다. 데이터 양이 엄청나게 많다면 (가령 몇 십만 이상) 어떻게 나눠지는지 큰 차이는 없을지도 모릅니다. 그런데 반도체 관련 데이터는 보통 그렇게 많지 않습니다. 데이터 양이 적으면 적을수록 training/validation을 나누어 보는 것을 더 많이 해보아야지 실질적인 generalization 성능을 가늠할 수 있습니다. 

K-fold cross validation은 그 방법 중 하나입니다. 데이터 셋을 특정 개수로 쪼갠 뒤(가령 다섯 개) Training/Validation을 반복 수행하는 것입니다. 반복 수행해서 나온 값들을 평균해서 validation 성능을 측정합니다. K-fold에서 k는 데이터 셋을 k 개로 쪼갠다는 의미입니다. Cross validation은 겹치지 않도록 교차 검증을 한다는 뜻입니다.  

아래는 scikit-learn을 이용한 예시입니다.

In [43]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor

- 보스턴 집값 데이터 함수를 불러옵니다.
- Scaling을 위해 Scaler를 불러옵니다. 
- k-fold 방식으로 index를 나눠주는 KFold와 shuffle split 방식으로 나누어주는 Shuffle Split을 불러옵니다. 
- Neural Network(다른 말로 multi-layer perceptron) Regressor를 불러옵니다. 

In [44]:
x, y = load_boston(return_X_y=True)  # 보스턴 집값 데이터를 불러옵니다. 

x_scaler = MinMaxScaler(feature_range=(-1, 1))  # -1과 1 사이로 x를 scaling하기 위해 scaler를 생성합니다. 
y_scaler = MinMaxScaler(feature_range=(-1, 1))  # -1과 1 사이로 y를 생성하기 위해 scaler를 생성합니다. 

y = np.reshape(y, (-1, 1))  # scaling 시 에러를 예방하기 위해 y의 차원을 1차원에서 2차원으로 변경합니다.
x_scaled = x_scaler.fit_transform(x)  # x를 scaling 합니다. 
y_scaled = y_scaler.fit_transform(y)  # y를 scaling 합니다. 

보스턴 집값 데이터를 부른 후 scaling합니다. MLPRegressor에 데이터를 입력하기 전에는 반드시 scaling을 해야 합니다. 앞선 tutorial에서는 training set과 test set을 나눈 후 training set에 맞춰 scaling 했지만 여기서는 편의를 위해 나누기 전에 scaling 하겠습니다. 결과상 큰 차이를 주지 않습니다. 

In [54]:
regressor = MLPRegressor()  # MLPRegressor instance를 생성하여 regressor를 정의합니다. 

kfold = KFold(n_splits=5, shuffle=True)  # k-fold cross validation을 위해 KFold 클래스의 instance를 생성합니다. 
val_scores = []  # 각 fold에서 계산한 score를 저장하기 위해 빈 리스트를 만듭니다. 

for train_index, val_index in kfold.split(x):  # KFold의 split method를 이용해 train_index와 val_index를 generation 합니다. 
    x_train = x_scaled[train_index]  # train_index로 x_scaled를 indexing 하여 x_train 데이터를 생성합니다. 
    y_train = y_scaled[train_index]  # train_index로 y_scaled를 indexing 하여 y_train 데이터를 생성합니다. 
    x_val = x_scaled[val_index]  # test_index로 x_scaled를 indexing 하여 x_val 데이터를 생성합니다. 
    y_val = y_scaled[val_index]  # test_index로 y_scaled를 indexing 하여 y_val 데이터를 생성합니다. 
    
    regressor.fit(x_train, y_train.ravel())  # training 데이터를 학습합니다.
    val_score = regressor.score(x_val, y_val.ravel())  # validation 데이터로 r-squre를 구합니다. 
    val_scores.append(val_score)  # 리스트에 저장합니다. 
    
val_scores = np.array(val_scores)  # 리스트를 numpy array로 바꿉니다.
val_score_mean = np.mean(val_scores)  # 평균 R-squre를 구합니다. 
print('Validation Score: {:.2f}'.format(val_score_mean)) 

Validation Score: 0.75


이번 tutorial에서는 Random Forest 대신에 Multi-layer perceptron 즉, Neural Network를 사용했습니다. MPLRegressor는 hyper-parameter에 따라 성능 차이가 심한 알고리즘입니다. 우선은 default hyper-parameter를 사용하겠습니다. 

KFold는 K-fold cross validation을 할 수 있도록 index를 generating 하는 클래스입니다. instance를 생성할 때 shuffle parameter는 True로 두는 것이 좋습니다. `split`이라는 method는 차례차례 training과 validation용 데이터를 나눌 수 있도록 index를 내뱉어주죠. 위처럼 for문에 넣어서 사용하면 됩니다. 

`n_splits`은 5라고 지정했기 때문에 `for`문은 총 5회 돌면서 training set과 validation set을 나눕니다. 그 각각에 대해 학습을 하고 validation score를 계산해서 저장합니다. 

`for`문이 끝나면 저장된 validation score 값들을 평균해 최종적인 validation score를 구합니다. 평균 값(mean)을 쓰기도 하고 중심 값(median)을 쓰기도 합니다. 

Validation score가 random forest일때보다 많이 낮네요. 아직 tuning이 덜 되었기 때문입니다. 

### cross_val_score 함수 이용하기 

Cross validation은 여러 알고리즘 및 hyper-parameter를 테스트해서 그중 베스트를 구하기 위한 목적입니다. 그런데 매번 위 코드와 같이 길게 적어야 한다면 테스트 하기가 번거롭겠죠. 전체 과정을 함수로 만들어 놓으면 편할 것 같습니다. scikit-learn에서 이미 만들어놨습니다. 

In [52]:
from sklearn.model_selection import cross_val_score

regressor = MLPRegressor()

cv = KFold(n_splits=5, shuffle=True)
scores = cross_val_score(regressor, x_scaled, y_scaled.ravel(), cv=cv)
print(scores)

[ 0.63035193  0.65494982  0.62250152  0.71651776  0.71545591]


regessor와 데이터를 입력하고 k 값을 입력하면 알아서 k-fold를 수행해 validation scores를 numpy array 형태로 출력합니다. 

In [53]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=1000)

cv = KFold(n_splits=5, shuffle=True)
scores = cross_val_score(rf, x_scaled, y_scaled.ravel(), cv=cv)
print(scores)

[ 0.81427955  0.86855085  0.90640846  0.89616404  0.79385097]


regressor를 random forest로 바꿔서 넣어봤습니다. 아까보다 훨씬 쉽게 테스트가 가능합니다. 

데이터가 적을 때 제대로 generalization 성능을 평가하려면 5-fold 정도로는 부족합니다. 훨씬 많이 시도해보는 것이 좋습니다. 데이터가 많을 때는 k-fold의 k 값을 높이거나 k-fold를 여러번 수행하면 되겠죠. 여러번 split 할 때는 scikit-learn에서 제공하는 `RepeatedKFold` 클래스를 이용하시면 됩니다. 

### Shuffle Split

K-fold cross validation 대신 Shuffle Split을 사용하셔도 됩니다. K-fold에서 k 값을 늘렸을 때 validation set 숫자가 너무 작아지면 validation score 값의 변이(variation)이 너무 크게 되어서 제대로 된 평가를 방해할 가능성도 있습니다. 이때는 shuffle split을 쓰시면 됩니다. 

In [51]:
from sklearn.model_selection import ShuffleSplit

ss = ShuffleSplit(n_splits=20, test_size=0.2)
rf = RandomForestRegressor(n_estimators=500)

scores = cross_val_score(rf, x_scaled, y_scaled.ravel(), cv=ss)
print(scores)
print('Median scores: {:.2f}'.format(np.median(scores)))

[ 0.85976802  0.82771327  0.83360453  0.88945236  0.88587958  0.88822028
  0.85721048  0.69252472  0.81635565  0.88239337  0.83678493  0.84533102
  0.90022859  0.85194058  0.88733445  0.86136759  0.88577278  0.84863407
  0.77534619  0.90836585]
Median scores: 0.86


shuffle split은 test_size만큼 random 하게 추출하는 작업을 n_splits 만큼 반복합니다. 위 코드에서는 cv 자리에 수자 대신 `ShuffleSplit`의 instance를 입력하였습니다. 입력한 instance가 내부적으로 돌면서 반복적인 validation을 수행합니다. 

### Group Split

반도체 데이터는 웨이퍼 단위로 데이터가 생성되는 경우가 많습니다. training과 validation set 혹은 test set을 나눌 때 웨이퍼 별로 나누면 좋겠죠. 이때는 group split 기능을 쓰시면 됩니다.

`GroupKFold`와 `GroupShuffleSplit`은 `KFold`와 `ShuffleSplit`과 동일하지만 같은 그룹 내 멤버는 항상 붙어있는 상태로 training과 validation을 쪼갭니다. 이때 그룹 label(가령 웨이퍼 숫자)는 numpy array 형태로 `split` 시 함께 전달되어야 합니다. 

더 쉬운 방법은 cross_val_score 시 그룹 label을 함께 입력하면 자동으로 group split을 수행합니다. 

In [58]:
from sklearn.model_selection import GroupShuffleSplit

ss = GroupShuffleSplit(n_splits=5, test_size=0.2)  # Group Shuffle Split을 정의합니다. 
rf = RandomForestRegressor(n_estimators=500)  

wafer_ids = np.random.randint(low=1, high=9, size=(x.shape[0]))  # 가상의 wafer id를 생성합니다. 

scores = cross_val_score(rf, x_scaled, y_scaled.ravel(), cv=ss, groups=wafer_ids)  # groups에 wafer id를 입력합니다. 

### cross_validate 사용하기 

`cross_val_score`는 validation score 값만 반환하죠. training score나 학습 시간, 추론 시간까지 모두 알고 싶다면, `cross_validate`를 사용하시면 됩니다. 

`cross_val_score`는 dictionary 형태로 반환하는데, 'test_score', train_score', 'fit_time', 'score_time'을 key로 입력하면 각각 validation score, training score, 학습 시간, validation set에 대한 추론 시간을 출력해줍니다. 

자세한 내용은 scikit-learn 매뉴얼을 참조 바랍니다. 